In [35]:
import os
import json
import pandas as pd
import traceback

In [36]:
from langchain.chat_models import ChatOpenAI

In [37]:
from dotenv import load_dotenv
load_dotenv()

True

In [38]:
api_key=os.getenv("OPENAI_API_KEY")

In [39]:
llm=ChatOpenAI(openai_api_key=api_key,model_name="gpt-3.5-turbo", temperature=0.5)

In [40]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [41]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

In [42]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [43]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [44]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [45]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [46]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [47]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [48]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [49]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [50]:
file_path=r"C:\Users\likit\mcqgen\data.txt"

In [51]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [52]:
print(TEXT)
TEXT='''Cellular Respiration in Eukaryotic Cells
Cellular respiration is a set of metabolic processes that occur within cells to convert biochemical energy from nutrients into adenosine triphosphate (ATP), and then release waste products. It is a critical process for the survival of eukaryotic cells, encompassing a series of reactions that are classified into three main stages: glycolysis, the citric acid cycle (Krebs cycle), and oxidative phosphorylation.

Glycolysis
Glycolysis is the first step in the breakdown of glucose to extract energy for cellular metabolism. This process takes place in the cytoplasm and involves the conversion of one molecule of glucose (a six-carbon compound) into two molecules of pyruvate (a three-carbon compound). Glycolysis can be divided into two phases:

Energy Investment Phase: Involves the expenditure of two ATP molecules to phosphorylate glucose and its intermediates.
Energy Payoff Phase: Produces four ATP molecules and two NADH molecules by substrate-level phosphorylation.
The net gain from glycolysis is two ATP molecules and two NADH molecules per glucose molecule.

Citric Acid Cycle (Krebs Cycle)
The pyruvate molecules produced in glycolysis are transported into the mitochondria, where they are converted into acetyl-CoA. The acetyl-CoA then enters the citric acid cycle, a series of enzyme-catalyzed chemical reactions that are crucial for the cellular metabolic process. The main functions of the citric acid cycle include:

The complete oxidation of acetyl-CoA to carbon dioxide.
The reduction of NAD+ and FAD to NADH and FADH2.
The production of one ATP (or GTP) per cycle through substrate-level phosphorylation.
The citric acid cycle generates three NADH and one FADH2 per acetyl-CoA molecule, which will be used in the next stage for ATP production.'''

In [53]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [54]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [55]:
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Cellular Respiration in Eukaryotic Cells
Cellular respiration is a set of metabolic processes that occur within cells to convert biochemical energy from nutrients into adenosine triphosphate (ATP), and then release waste products. It is a critical process for the survival of eukaryotic cells, encompassing a series of reactions that are classified into three main stages: glycolysis, the citric acid cycle (Krebs cycle), and oxidative phosphorylation.

Glycolysis
Glycolysis is the first step in the breakdown of glucose to extract energy for cellular metabolism. This process takes place in the cytoplasm and involves the conversion of one molecule of glucose (a six-carbon compound) into two molecules of pyruvate (a three-carbon compound). Glycolysis can be divided into two phases:

Energy Investment Phase: Involves the expenditure of two ATP molecules to phosphorylate glucose and its 

In [61]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2018
Prompt Tokens:1304
Completion Tokens:714
Total Cost:0.0033840000000000003


In [62]:
response

{'text': 'Cellular Respiration in Eukaryotic Cells\nCellular respiration is a set of metabolic processes that occur within cells to convert biochemical energy from nutrients into adenosine triphosphate (ATP), and then release waste products. It is a critical process for the survival of eukaryotic cells, encompassing a series of reactions that are classified into three main stages: glycolysis, the citric acid cycle (Krebs cycle), and oxidative phosphorylation.\n\nGlycolysis\nGlycolysis is the first step in the breakdown of glucose to extract energy for cellular metabolism. This process takes place in the cytoplasm and involves the conversion of one molecule of glucose (a six-carbon compound) into two molecules of pyruvate (a three-carbon compound). Glycolysis can be divided into two phases:\n\nEnergy Investment Phase: Involves the expenditure of two ATP molecules to phosphorylate glucose and its intermediates.\nEnergy Payoff Phase: Produces four ATP molecules and two NADH molecules by s

In [63]:
quiz=response.get("quiz")

In [64]:
quiz=json.loads(quiz)

In [65]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [66]:
quiz_table_data

[{'MCQ': 'Which stage of cellular respiration involves the conversion of glucose into pyruvate?',
  'Choices': 'a: Glycolysis | b: Citric Acid Cycle | c: Oxidative Phosphorylation | d: Krebs Cycle',
  'Correct': 'a'},
 {'MCQ': 'Where does glycolysis take place within the cell?',
  'Choices': 'a: Mitochondria | b: Nucleus | c: Cytoplasm | d: Endoplasmic Reticulum',
  'Correct': 'c'},
 {'MCQ': 'Which phase of glycolysis involves the expenditure of ATP molecules?',
  'Choices': 'a: Energy Investment Phase | b: Energy Payoff Phase | c: Citric Acid Phase | d: Oxidative Phase',
  'Correct': 'a'},
 {'MCQ': 'What is the main function of the citric acid cycle?',
  'Choices': 'a: Conversion of glucose to pyruvate | b: Production of ATP through oxidative phosphorylation | c: Complete oxidation of acetyl-CoA to carbon dioxide | d: Transportation of pyruvate into the mitochondria',
  'Correct': 'c'},
 {'MCQ': 'How many NADH molecules are generated per acetyl-CoA molecule in the citric acid cycle?',

In [32]:
quiz=pd.DataFrame(quiz_table_data)

In [33]:
quiz.to_csv("machinelearning.csv",index=False)

In [34]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'05_31_2024_14_11_52'

In [67]:
response

{'text': 'Cellular Respiration in Eukaryotic Cells\nCellular respiration is a set of metabolic processes that occur within cells to convert biochemical energy from nutrients into adenosine triphosphate (ATP), and then release waste products. It is a critical process for the survival of eukaryotic cells, encompassing a series of reactions that are classified into three main stages: glycolysis, the citric acid cycle (Krebs cycle), and oxidative phosphorylation.\n\nGlycolysis\nGlycolysis is the first step in the breakdown of glucose to extract energy for cellular metabolism. This process takes place in the cytoplasm and involves the conversion of one molecule of glucose (a six-carbon compound) into two molecules of pyruvate (a three-carbon compound). Glycolysis can be divided into two phases:\n\nEnergy Investment Phase: Involves the expenditure of two ATP molecules to phosphorylate glucose and its intermediates.\nEnergy Payoff Phase: Produces four ATP molecules and two NADH molecules by s

In [69]:
print(response['review'])


{
"1": {
"mcq": "Which stage of cellular respiration involves the conversion of glucose into pyruvate?",
"options": {"a": "Glycolysis", "b": "Citric Acid Cycle", "c": "Oxidative Phosphorylation", "d": "Krebs Cycle"},
"correct": "a"
},
"2": {
"mcq": "Where does glycolysis take place within the cell?",
"options": {"a": "Mitochondria", "b": "Nucleus", "c": "Cytoplasm", "d": "Endoplasmic Reticulum"},
"correct": "c"
},
"3": {
"mcq": "What is the main function of the citric acid cycle?",
"options": {"a": "Convert acetyl-CoA to glucose", "b": "Produce ATP through oxidative phosphorylation", "c": "Oxidize acetyl-CoA to carbon dioxide", "d": "Convert pyruvate to lactate"},
"correct": "c"
},
"4": {
"mcq": "How many ATP molecules are produced in the energy payoff phase of glycolysis?",
"options": {"a": "2 ATP", "b": "4 ATP", "c": "6 ATP", "d": "8 ATP"},
"correct": "b"
},
"5": {
"mcq": "What is the net gain of ATP molecules from glycolysis per glucose molecule?",
"options": {"a": "0 ATP", "b": "2